In [1]:
%cd ..

/home/s4hoque/DiVAE


In [2]:
# Python imports
import os
import sys

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if par_dir not in sys.path:
    sys.path.append(par_dir)
    
# ML imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

# DiVAE imports
from models.rbm.chimeraRBM import ChimeraRBM
from models.rbm.qimeraRBM import QimeraRBM
from models.rbm.rbm import RBM
from models.samplers.pcd import PCD
from models.autoencoders.gumboltCaloCRBM import GumBoltCaloCRBM

from nbutils import *

# DWave imports
from dwave.system import DWaveSampler, LeapHybridSampler
import neal

[07:17:29.743] INFO   DiVAE                                             Willkommen!
[07:17:29.745] INFO   DiVAE                                             Loading configuration.
2022-09-22 07:17:30,745 dwave.cloud INFO  MainThread Log level for 'dwave.cloud' namespace set to 0
[07:17:30.745] INFO   dwave.cloud                                       Log level for 'dwave.cloud' namespace set to 0


In [3]:
ZERO = torch.tensor(0., dtype=torch.float)
MINUS_ONE = torch.tensor(-1., dtype=torch.float)

In [4]:
def init_model(cfg):
    model = GumBoltCaloCRBM(flat_input_size=[504],
                            train_ds_mean=0.,
                            activation_fct=torch.nn.ReLU(),
                            cfg=cfg)
    return model

In [5]:
with initialize(config_path="../configs"):
    cfg = compose(config_name="config")

/opt/conda/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [6]:
%pwd

'/home/s4hoque/DiVAE'

In [7]:
model = init_model(cfg)
model.create_networks()

2022-09-22 07:17:49,119 dwave.cloud.client.base INFO  MainThread Fetching available regions from the Metadata API at 'https://cloud.dwavesys.com/metadata/v1/'
[07:17:49.119] INFO   dwave.cloud.client.base                           Fetching available regions from the Metadata API at 'https://cloud.dwavesys.com/metadata/v1/'
2022-09-22 07:17:49,460 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[07:17:49.460] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-09-22 07:17:49,515 dwave.cloud.client.base INFO  MainThread Req

Initializing NetworkV3


In [8]:
"""
No need to run this for beta_eff* estimation
"""
# run_path = '/home/akajal/DiVAE/outputs/2022-03-11/15-21-04/wandb/run-20220311_152104-2junzvyl/files/GumBoltCaloCRBM_calo_default_latest.pth'
# load_state(model, run_path, 'cuda:{0}'.format(cfg.gpu_list[0]))

'\nNo need to run this for beta_eff* estimation\n'

In [9]:
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"}, token="DEV-3f2d995f7cc156db3ee4077a24fcc9fbb69f48ea")

aux_crbm = QimeraRBM(n_visible=model.prior._n_visible, n_hidden=model.prior._n_hidden)
aux_crbm_sampler = PCD(batch_size=1024, RBM=aux_crbm, n_gibbs_sampling_steps=100)

2022-09-22 07:17:57,720 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[07:17:57.720] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-09-22 07:17:57,786 dwave.cloud.client.base INFO  MainThread Requested a solver that best matches feature filters={}
[07:17:57.786] INFO   dwave.cloud.client.base                           Requested a solver that best matches feature filters={}
2022-09-22 07:17:57,790 dwave.cloud.client.base INFO  MainThread Fetching solvers according to filters={'topology__type': 'chimera', 'chip_id': 

In [55]:
def beta_fixed_qpu(num_iterations:int=10, lr:float=0.01, beta_init=10., n_reads:int=100, qpu_sampler=None, aux_crbm_sampler=None):
    """Estimate the temperature associated with a given QPU configuration
    
    :param num_iterations (int) : Number of iterations
    :param lr (float) : Learning rate
    :param beta_init (float) : Initial estimate of the QPU temperature
    :param n_qpu_samples (int) : Number of QPU samples
    :param qpu_sampler
    
    """
    assert qpu_sampler is not None
    assert aux_crbm_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    # Auxiliary scaled RBM
    aux_crbm = aux_crbm_sampler.rbm
    aux_crbm_edgelist = aux_crbm.pruned_edge_list
    
    # Setup QPU control variables
    n_vis = len(aux_crbm.visible_qubit_idxs)
    n_hid = len(aux_crbm.visible_qubit_idxs)
    qubit_idxs = aux_crbm.visible_qubit_idxs+aux_crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(aux_crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(aux_crbm.hidden_qubit_idxs)}
        
        
    '''
    We generate an Ising model with random weights and biases
    Weights(J) are in [-1.5,0.5]
    Biases(h) are in [-2,2]
    '''
    #wlim = [-1,1]
    #ising_weights = torch.nn.Parameter((wlim[1]-wlim[0])*torch.rand(n_vis, n_hid) + wlim[0], requires_grad=False)
    ising_weights = torch.nn.Parameter(-2*torch.rand(n_vis, n_hid) + 1.5, requires_grad=False)
    
    """
    Also note that we are using a Chimera RBM architecture. Thus we have to transform 
    our Ising model by modifying the couplings to get the Chimera RBM architecture.
    We do this by masking the corresponding RBM weights of our ising_model
    and then reverting the rbm model back to the ising model. 
    """
    aux_crbm_weights = 4.*ising_weights # weights of the corresponding RBM
    aux_crbm_weights = aux_crbm_weights*aux_crbm.weights_mask # mask to get Chimera RBM architecture
    ising_weights = aux_crbm_weights*0.25 # revert back to ising model
    
    hlim = [-2,2] # for now
    ising_vbias = torch.nn.Parameter((hlim[1]-hlim[0])*torch.rand(n_vis)+hlim[0], requires_grad=False)
    ising_hbias = torch.nn.Parameter((hlim[1]-hlim[0])*torch.rand(n_hid)+hlim[0], requires_grad=False)
    
    dwave_weights_np = -ising_weights.detach().cpu().numpy()
    print("J range = ({0}, {1})".format(np.min(dwave_weights_np),
                                        np.max(dwave_weights_np)))
    
    h = {qubit_idx:0. for qubit_idx in qubit_idxs}
    J = {}
    for edge in aux_crbm_edgelist:
        if edge[0] in aux_crbm.visible_qubit_idxs:
            J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
        else:
            J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
                
    response = qpu_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
    dwave_samples, dwave_energies = batch_dwave_samples(response)
    dwave_samples = torch.tensor(dwave_samples, dtype=torch.float)
    dwave_vis, dwave_hid = dwave_samples[:, :n_vis], dwave_samples[:, n_vis:]
        
    dwave_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, dwave_vis, dwave_hid)
    dwave_energy_exp = torch.mean(dwave_energy_exp, axis=0)
    print("dwave_energy_exp : {0}".format(dwave_energy_exp))
    
    """
    Now recall we already have the weights of the auxilliary RBM from our previous work.
    We now only need to convert the ising biases to RBM biases.
    """
    aux_crbm._visible_bias = torch.nn.Parameter(2.*(ising_vbias - torch.sum(ising_weights, dim=1)), requires_grad=False)
    aux_crbm._hidden_bias = torch.nn.Parameter(2.*(ising_hbias - torch.sum(ising_weights, dim=0)), requires_grad=False)
    
    """
    Recall that in each iteration of the beta_eff* estimation we change the RBM parameters. However
    we still sample from the same RBM which has been initialized above. Hence, we keep track 
    of the original parameters by the following:
    """
    rbm_orig_vis = aux_crbm._visible_bias
    rbm_orig_hid = aux_crbm._hidden_bias
    rbm_orig_weights = aux_crbm_weights
    
    for i in range(num_iterations):
        # Calculate the rescaled parameters of the auxiliary RBM
        aux_crbm.weights = aux_crbm_weights/beta
        aux_crbm._visible_bias = torch.nn.Parameter(aux_crbm._visible_bias/beta, requires_grad=False) 
        aux_crbm._hidden_bias = torch.nn.Parameter(aux_crbm._hidden_bias/beta, requires_grad=False)
        
        
        aux_crbm_sampler.rbm = aux_crbm
        
        aux_crbm_vis, aux_crbm_hid = aux_crbm_sampler.block_gibbs_sampling()
        
        """
        Finding energy using RBM method
        """
        ising_energy_rbm_params = ising_energy_rbm(rbm_orig_weights, rbm_orig_vis, rbm_orig_hid, aux_crbm_vis, aux_crbm_hid)
        rbm_energy_exp = torch.mean(ising_energy_rbm_params, axis=0)
        
        
        aux_crbm_vis = torch.where(aux_crbm_vis == ZERO, MINUS_ONE, aux_crbm_vis)
        aux_crbm_hid = torch.where(aux_crbm_hid == ZERO, MINUS_ONE, aux_crbm_hid)
        
        aux_crbm_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, aux_crbm_vis, aux_crbm_hid)
        aux_crbm_energy_exp = torch.mean(aux_crbm_energy_exp, axis=0)
        
        print("aux_crbm_energy_exp : {0}, rbm_exp : {1}, beta : {2}".format(aux_crbm_energy_exp, rbm_energy_exp, beta))
        beta = beta - lr*(-float(aux_crbm_energy_exp)+float(dwave_energy_exp))
        betas.append(beta)
        
    return betas

In [58]:
betas = beta_fixed_qpu(num_iterations=100,
                       lr=0.0001,
                       beta_init=8.,
                       n_reads=100,
                       qpu_sampler=qpu_sampler,
                       aux_crbm_sampler=aux_crbm_sampler)

J range = (-1.4994338750839233, 0.49997055530548096)
dwave_energy_exp : 498.97686767578125
aux_crbm_energy_exp : 749.8893432617188, rbm_exp : 749.8892211914062, beta : 8.0
aux_crbm_energy_exp : 1057.1163330078125, rbm_exp : 1057.1163330078125, beta : 8.025091247558594
aux_crbm_energy_exp : 1121.641357421875, rbm_exp : 1121.641357421875, beta : 8.080905194091796
aux_crbm_energy_exp : 1118.627197265625, rbm_exp : 1118.626953125, beta : 8.143171643066406
aux_crbm_energy_exp : 1108.3907470703125, rbm_exp : 1108.3905029296875, beta : 8.20513667602539
aux_crbm_energy_exp : 1095.786865234375, rbm_exp : 1095.786865234375, beta : 8.266078063964843
aux_crbm_energy_exp : 1082.432373046875, rbm_exp : 1082.432373046875, beta : 8.325759063720701
aux_crbm_energy_exp : 1070.8267822265625, rbm_exp : 1070.8267822265625, beta : 8.384104614257812
aux_crbm_energy_exp : 1062.8834228515625, rbm_exp : 1062.8834228515625, beta : 8.441289605712889
aux_crbm_energy_exp : 1055.784423828125, rbm_exp : 1055.78442382

aux_crbm_energy_exp : 703.9413452148438, rbm_exp : 703.9413452148438, beta : 11.092127642822266
aux_crbm_energy_exp : 705.8516235351562, rbm_exp : 705.851806640625, beta : 11.112624090576173
aux_crbm_energy_exp : 700.4968872070312, rbm_exp : 700.496826171875, beta : 11.13331156616211
aux_crbm_energy_exp : 699.5419921875, rbm_exp : 699.5419311523438, beta : 11.153463568115235
aux_crbm_energy_exp : 696.8995971679688, rbm_exp : 696.8994140625, beta : 11.173520080566407
aux_crbm_energy_exp : 695.0702514648438, rbm_exp : 695.0701293945312, beta : 11.193312353515626
aux_crbm_energy_exp : 692.5932006835938, rbm_exp : 692.5931396484375, beta : 11.212921691894532
aux_crbm_energy_exp : 689.3853149414062, rbm_exp : 689.3853149414062, beta : 11.232283325195313
aux_crbm_energy_exp : 685.7107543945312, rbm_exp : 685.7105712890625, beta : 11.251324169921876
aux_crbm_energy_exp : 684.8931884765625, rbm_exp : 684.8931274414062, beta : 11.26999755859375
aux_crbm_energy_exp : 683.94482421875, rbm_exp : 6

In [ ]:
plot_betas(betas)